In [12]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import pathlib
from datetime import datetime
from tensorflow import keras
logdir = "logs/scalars/" + "Pure_Classifer" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from sklearn.model_selection import train_test_split

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)


In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

Name: /physical_device:GPU:0   Type: GPU


In [5]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_data_dir = "./Dataset"
img_height=224
img_width=224
batch_size=1
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=2401,
    class_mode='categorical',
    subset='training',
    seed=42
)

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=600,
    class_mode='categorical',
    subset='validation',
    seed=42
)

Found 2401 images belonging to 5 classes.
Found 600 images belonging to 5 classes.


In [6]:
x, y = train_generator.next()
x_val, y_val = validation_generator.next()

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:

model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=5, activation="softmax"))

In [14]:
from tensorflow.keras.optimizers import Adam
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


In [16]:
model.fit(x_train, y_train,
          batch_size=32,
          epochs=40,
          validation_data=(x_val, y_val),
          shuffle=True,
          callbacks=[tensorboard_callback]
         )

Train on 1920 samples, validate on 600 samples
Epoch 1/40
1920/1920 [==============================] - 12s 6ms/sample - loss: 1.6347 - accuracy: 0.2052 - val_loss: 1.6104 - val_accuracy: 0.2000
Epoch 2/40
1920/1920 [==============================] - 10s 5ms/sample - loss: 1.6131 - accuracy: 0.2823 - val_loss: 1.4035 - val_accuracy: 0.3817
Epoch 3/40
1920/1920 [==============================] - 10s 5ms/sample - loss: 1.5098 - accuracy: 0.3719 - val_loss: 1.4198 - val_accuracy: 0.4617
Epoch 4/40
1920/1920 [==============================] - 10s 5ms/sample - loss: 1.2952 - accuracy: 0.4719 - val_loss: 1.2911 - val_accuracy: 0.4583
Epoch 5/40
1920/1920 [==============================] - 10s 5ms/sample - loss: 1.1714 - accuracy: 0.5453 - val_loss: 1.0709 - val_accuracy: 0.6067
Epoch 6/40
1920/1920 [==============================] - 10s 5ms/sample - loss: 1.0388 - accuracy: 0.5922 - val_loss: 1.0845 - val_accuracy: 0.5250
Epoch 7/40
1920/1920 [==============================] - 10s 5ms/sample 

# 